todo: 
- letní filmy, co už nedávají (víno)
- záhada hlavolamu je seriál

In [2]:
import os
import polars as pl

In [3]:
pl.Config(tbl_rows=200)

In [4]:
from calendar import monthrange

In [5]:
def monthday(date_value):
    # Here date_value is a datetime.datetime object, not a Polars column
    month_length = monthrange(date_value.year, date_value.month)[1]
    return date_value.month + (date_value.day / month_length)

In [6]:
df = pl.read_parquet(
    os.path.join("data","fdb.parquet")
).with_columns(
    pl.col("datum").dt.month().alias("mesic")
).with_columns(
    pl.col("datum").dt.week().alias("tyden")
).with_columns(
    pl.col("datum").dt.year().alias("rok")
).with_columns(
    pl.when(
        pl.col("url").str.contains("serial/")
    ).then(
        pl.lit("serial")
    ).when(
        pl.col("url").str.contains("film/")
    ).then(
        pl.lit("film")
    ).alias(
        "kategorie"
    )
).filter(
    pl.col("rok").is_between(2015,2024)
).sort(
    by="datum"
)

sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


In [7]:
df.head(2)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""Prima""",2015-01-01 00:00:00,"""Děsnej doják""","""31.12.2019 v 00:10 (RTL2)""","""Další díl/opakování:31.12.2019…","""film/desnej-dojak-date-movie/4…",1,1,2015,"""film"""
"""Nova Cinema""",2015-01-01 00:00:00,"""PF 2015""",null,null,null,1,1,2015,null


In [8]:
df.tail(2)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""Prima Show""",2024-12-31 23:50:00,"""Tyhle šaty beru! XI""","""18.01.2025 v 23:10 (Prima Show…","""Další díl/opakování:18.01.2025…","""film/tyhle-saty-beru-say-yes-t…",12,1,2024,"""film"""
"""Prima Cool""",2024-12-31 23:50:00,"""Simpsonovi XXXIII""","""20.01.2025 v 13:15 (Prima Cool…","""Další díl/opakování:20.01.2025…","""serial/simpsonovi-the-simpsons…",12,1,2024,"""serial"""


In [9]:
df.sample(10).select(pl.col("noticka")).to_series().to_list()

['Další díl/opakování:12.07.2024 v 22:25 (NOVA)15.07.2024 v 18:05 (Nova Cinema)16.07.2024 v 01:05 (Nova Action)16.07.2024 v 10:20 (Nova Cinema)',
 'Další díl/opakování:02.12.2023 v 15:15 (Cinemax2)03.12.2023 v 04:05 (Cinemax2)',
 'Další díl/opakování:15.04.2025 v 16:40 (Nova Action)16.04.2025 v 16:40 (Nova Action)17.04.2025 v 16:40 (Nova Action)18.04.2025 v 16:40 (Nova Action)19.04.2025 v 16:40 (Nova Action)20.04.2025 v 16:40 (Nova Action)21.04.2025 v 02:00 (Nova Action)22.04.2025 v 16:40 (Nova Action)',
 'Další díl/opakování:12.02.2024 v 10:50 (NOVA)13.02.2024 v 10:50 (NOVA)14.02.2024 v 10:50 (NOVA)15.02.2024 v 10:50 (NOVA)16.02.2024 v 10:50 (NOVA)',
 'Velká silvestrovská revue. Účinkují: V. Dvořák, J. Bohdalová, V. Menšík, F. Filipovský, L. Sobota, J. Sovák, M. Kopecký, P. Černocká, H. Vondráčková, J. Korn, W. Matuška, K. Gott, H. Zagorová, N. Urbánková a další.',
 None,
 'Další díl/opakování:08.09.2021 v 01:05 (AMC)',
 'Další díl/opakování:21.01.2025 v 01:20 (TV Barrandov)21.01.2025

In [10]:
kanaly = df.select(pl.col("kanal")).unique().to_series().to_list()
kanaly

['CS Film',
 'Film Europe Channel',
 'ČT2',
 'Cinemax',
 'Nova Lady',
 'Nova Action',
 'Prima Show',
 'Prima',
 'Film+',
 'Prima LOVE',
 'Nova 2',
 'ČT1',
 'Cinemax2',
 'NOVA',
 'Prima STAR',
 'Prima Cool',
 'TV Barrandov',
 'Nova Cinema',
 'Nova Fun',
 'CNN Prima News',
 'VOYO',
 'ČT 3']

In [11]:
verejnopravni = [k for k in kanaly if 'ČT' in k]
print(verejnopravni)
soukrome = [k for k in kanaly if 'ČT' not in k]
print(soukrome)

['ČT2', 'ČT1', 'ČT 3']
['CS Film', 'Film Europe Channel', 'Cinemax', 'Nova Lady', 'Nova Action', 'Prima Show', 'Prima', 'Film+', 'Prima LOVE', 'Nova 2', 'Cinemax2', 'NOVA', 'Prima STAR', 'Prima Cool', 'TV Barrandov', 'Nova Cinema', 'Nova Fun', 'CNN Prima News', 'VOYO']


In [12]:
vysilane_verejnopravni = df.filter(pl.col("kanal").is_in(verejnopravni)).select(pl.col("nazev")).unique().to_series().to_list()
vysilane_soukromymi = df.filter(pl.col("kanal").is_in(soukrome)).select(pl.col("nazev")).unique().to_series().to_list()
vysilane_vsude = [x for x in vysilane_verejnopravni if x in vysilane_soukromymi]

In [107]:
aspon_na_dvou = df.group_by("nazev").agg(pl.col("kanal").unique().len()).filter(pl.col("kanal") >= 2).select(pl.col("nazev")).to_series().to_list()

In [13]:
x_let = df.with_columns(pl.col('datum').dt.year().alias('rok')).group_by('nazev').agg(pl.col('rok').unique().len()).filter(pl.col('rok') >= 5).select(pl.col('nazev')).unique().to_series().to_list()

In [109]:
vsude_a_x_let = [x for x in x_let if x in aspon_na_dvou]

In [111]:
zajima_nas = ['ČT1', 'ČT2', 'NOVA', 'Prima']

In [112]:
letni = df.filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(vsude_a_x_let) & pl.col('mesic').is_in([7,8])).group_by('nazev').len().rename({'len':'v_lete'})

In [113]:
zimni = df.filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(vsude_a_x_let) & pl.col('mesic').is_in([1,2,3,10,11,12])).group_by('nazev').len().rename({'len':'v_zime'})

In [114]:
zimni = df.filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(vsude_a_x_let) & (~pl.col('mesic').is_in([7,8]))).group_by('nazev').len().rename({'len':'v_zime'})

In [115]:
kategorie = df.filter(pl.col("nazev").is_in(vsude_a_x_let)).group_by("nazev").agg(pl.col("kategorie").mode()).explode("kategorie").drop_nulls()
kategorie = df.filter(pl.col("nazev").is_in(vsude_a_x_let)).group_by(['nazev','kategorie']).len().sort(by='len',descending=True).unique(subset=['nazev'], keep='first').drop('len').drop_nulls()
kategorie

nazev,kategorie
str,str
"""Pepa""","""film"""
"""Akce Býčí oko""","""film"""
"""Iluzionista""","""film"""
"""Poslední skaut""","""film"""
"""Osamělý mstitel""","""film"""
"""Mladí muži za pultem""","""film"""
"""Návrat do budoucnosti""","""serial"""
"""Slunečná""","""serial"""
"""Příběhy o lásce: Tajemství rod…","""film"""


In [116]:
pomer = letni.join(zimni, how='full', on='nazev').with_columns(
    pl.when(
        pl.col('nazev').is_null()
    ).then(
        pl.col('nazev_right')
    ).otherwise(
        pl.col('nazev')
    ).alias(
        'nazev'
    )
).with_columns(
    pl.col('v_lete').fill_null(0)
).drop(
    'nazev_right'
).with_columns(
    (pl.col('v_lete') / pl.col('v_zime')).alias('pomer')
)

In [117]:
pomer.filter(pl.col("nazev").str.contains("Lepší už"))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Lepší už to nebude""",5,1,5.0


In [118]:
nechceme = [
    "Šaráda",
    "Záchranáři",
    "Máma",
    "Helena",
    "Hodinový manžel"
]

In [119]:
filmy = kategorie.filter(pl.col('kategorie') == 'film').filter(~pl.col('nazev').is_in(nechceme)).select(pl.col('nazev')).to_series().to_list()
serialy = kategorie.filter(pl.col('kategorie') == 'serial').filter(~pl.col('nazev').is_in(nechceme)).select(pl.col('nazev')).to_series().to_list()

In [120]:
pomer.filter(pl.col('nazev').str.contains('Hříšný t'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Hříšný tanec""",4,9,0.444444
"""Hříšný tanec 2""",0,1,0.0


In [121]:
letni_filmy = pomer.filter(pl.col('nazev').is_in(filmy)).filter(pl.col('v_lete') >= 5).sort(by='pomer',descending=True).with_columns(pl.lit("film").alias("kategorie")).head(50)
letni_filmy

nazev,v_lete,v_zime,pomer,kategorie
str,u32,u32,f64,str
"""Indiana Jones a Poslední křížo…",13,null,null,"""film"""
"""Dneska to konečně rozjedeme""",5,null,null,"""film"""
"""Pasiáns""",5,null,null,"""film"""
"""Pinocchiova dobrodružství""",26,null,null,"""film"""
"""Pohádky z lesa""",7,null,null,"""film"""
"""Indiana Jones a dobyvatelé ztr…",13,null,null,"""film"""
"""Indiana Jones a Království kři…",13,null,null,"""film"""
"""Turner a Hooch""",5,null,null,"""film"""
"""Indiana Jones a Chrám zkázy""",13,null,null,"""film"""


In [122]:
letni_serialy = pomer.filter(pl.col('nazev').is_in(serialy)).filter(pl.col('v_lete') >= 25).sort(by='pomer',descending=True).with_columns(pl.lit("seriál").alias("kategorie")).head(50)
letni_serialy

nazev,v_lete,v_zime,pomer,kategorie
str,u32,u32,f64,str
"""Tajemství proutěného košíku""",39,null,null,"""seriál"""
"""Malý pitaval z velkého města""",177,30,5.9,"""seriál"""
"""Případy pro Lauru""",64,16,4.0,"""seriál"""
"""Pan Tau""",61,20,3.05,"""seriál"""
"""Doktorka z Dixie""",85,34,2.5,"""seriál"""
"""Chalupáři""",123,54,2.277778,"""seriál"""
"""Kameňák""",63,29,2.172414,"""seriál"""
"""Návštěvníci""",26,13,2.0,"""seriál"""
"""Vinaři""",150,88,1.704545,"""seriál"""


In [198]:
letni_televize = pl.concat(
    [letni_filmy, letni_serialy]
).sort(by='pomer',descending=True).filter((pl.col('pomer') >= 2) | (pl.col('pomer').is_null()))

In [200]:
letni_televize

nazev,v_lete,v_zime,pomer,kategorie
str,u32,u32,f64,str
"""Indiana Jones a Poslední křížo…",13,null,null,"""film"""
"""Dneska to konečně rozjedeme""",5,null,null,"""film"""
"""Pasiáns""",5,null,null,"""film"""
"""Pinocchiova dobrodružství""",26,null,null,"""film"""
"""Pohádky z lesa""",7,null,null,"""film"""
"""Indiana Jones a dobyvatelé ztr…",13,null,null,"""film"""
"""Indiana Jones a Království kři…",13,null,null,"""film"""
"""Turner a Hooch""",5,null,null,"""film"""
"""Indiana Jones a Chrám zkázy""",13,null,null,"""film"""


In [125]:
pomer.filter(~pl.col('nazev').is_in(serialy) & ~pl.col('nazev').is_in(filmy)).filter(pl.col('v_lete') >= 50).sort(by='pomer',descending=True).head(20)

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Hodinový manžel""",51,18,2.833333
"""Máma""",85,35,2.428571
"""Záchranáři""",54,31,1.741935
"""Helena""",113,95,1.189474
"""Žiješ jenom 2x""",251,630,0.398413
"""V zajetí železné opony""",115,448,0.256696
"""Ta naša kapela""",96,397,0.241814
"""Zprávy v českém znakovém jazyc…",674,3097,0.21763
"""Zajímavosti z regionů""",342,1623,0.210721


In [126]:
df.filter(pl.col("nazev") == "Kojak")

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT1""",2015-01-05 15:15:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",1,2,2015,"""serial"""
"""ČT1""",2015-01-06 15:15:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",1,2,2015,"""serial"""
"""ČT1""",2015-01-07 15:15:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",1,2,2015,"""serial"""
"""ČT1""",2015-01-08 15:15:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",1,2,2015,"""serial"""
"""ČT1""",2015-01-12 15:05:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",1,3,2015,"""serial"""
"""ČT1""",2015-01-13 15:05:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",1,3,2015,"""serial"""
"""ČT1""",2015-01-14 15:05:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",1,3,2015,"""serial"""
"""ČT1""",2015-01-15 15:05:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",1,3,2015,"""serial"""
"""ČT1""",2015-01-19 15:05:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",1,4,2015,"""serial"""


In [127]:
kategorie

nazev,kategorie
str,str
"""Pepa""","""film"""
"""Akce Býčí oko""","""film"""
"""Iluzionista""","""film"""
"""Poslední skaut""","""film"""
"""Osamělý mstitel""","""film"""
"""Mladí muži za pultem""","""film"""
"""Návrat do budoucnosti""","""serial"""
"""Slunečná""","""serial"""
"""Příběhy o lásce: Tajemství rod…","""film"""


In [202]:
vyber_strojovy = letni_televize.filter(~pl.col('nazev').is_in(nechceme)).select(pl.col('nazev')).to_series().to_list()
graf = df.filter(pl.col('nazev').is_in(vyber_strojovy)).filter(pl.col("kanal").is_in(zajima_nas)).group_by(['nazev','tyden']).len().join(
    kategorie, on="nazev", how="left"
)

graf.sort(by=['nazev','tyden'])

nazev,tyden,len,kategorie
str,i8,u32,str
"""2012""",6,1,"""film"""
"""2012""",17,1,"""film"""
"""2012""",26,2,"""film"""
"""2012""",28,1,"""film"""
"""2012""",33,2,"""film"""
"""2Bobule""",29,1,"""film"""
"""2Bobule""",30,1,"""film"""
"""2Bobule""",32,2,"""film"""
"""2Bobule""",33,10,"""film"""


In [129]:
import altair as alt

In [130]:
df.sample(10)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""Nova 2""",2020-01-05 03:35:00,"""Will a Grace""","""13.12.2022 v 01:50 (Nova 2)14.…","""Další díl/opakování:13.12.2022…","""serial/will-a-grace-will-grace…",1,1,2020,"""serial"""
"""NOVA""",2023-08-13 05:55:00,"""Zig a Sharko""","""18.04.2025 v 06:05 (NOVA)20.04…","""Další díl/opakování:18.04.2025…","""serial/zig-a-sharko-zig-et-sha…",8,32,2023,"""serial"""
"""Cinemax2""",2017-06-12 11:00:00,"""Star Trek VI: Neobjevená země""","""18.10.2024 v 20:30 (Nova Actio…","""Další díl/opakování:18.10.2024…","""film/star-trek-vi-neobjevena-z…",6,24,2017,"""film"""
"""ČT2""",2020-03-08 06:40:00,"""Prasátko Peppa""","""15.04.2025 v 22:35 (Nick Jr.)1…","""Další díl/opakování:15.04.2025…","""serial/prasatko-peppa-peppa-pi…",3,10,2020,"""serial"""
"""ČT2""",2017-05-18 03:35:00,"""Odpad společnosti?""",null,"""Dokument (2008). Sexuální deli…",null,5,20,2017,null
"""Prima""",2021-05-02 18:52:00,"""Počasí""",null,null,null,5,17,2021,null
"""ČT2""",2019-04-01 17:25:00,"""Poslední tajemství třetí říše""","""(4/6)""","""Hitlerův architekt. Dokumentár…",null,4,14,2019,null
"""Nova 2""",2019-10-31 20:15:00,"""Dva a půl chlapa XII""","""18.01.2025 v 18:10 (Nova Fun)1…","""Další díl/opakování:18.01.2025…","""serial/dva-a-pul-chlapa-two-an…",10,44,2019,"""serial"""
"""ČT2""",2020-12-10 05:30:00,"""Nedej se""","""10.04.2025 v 04:50 (ČT2)13.04.…","""Další díl/opakování:10.04.2025…","""film/nedej-se/66967""",12,50,2020,"""film"""


In [131]:
df.filter(pl.col('nazev').str.contains('Praštěná'))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""NOVA""",2015-02-22 10:55:00,"""Praštěná holka""","""02.03.2025 v 13:15 (Paramount …","""Další díl/opakování:02.03.2025…","""film/prastena-holka-cluelless/…",2,8,2015,"""film"""
"""NOVA""",2015-12-29 14:05:00,"""Praštěná holka""","""02.03.2025 v 13:15 (Paramount …","""Další díl/opakování:02.03.2025…","""film/prastena-holka-cluelless/…",12,53,2015,"""film"""
"""Nova Cinema""",2015-12-30 07:40:00,"""Praštěná holka""","""18.01.2025 v 09:25 (NOVA)20.01…","""Další díl/opakování:18.01.2025…","""film/prastena-holka-cluelless/…",12,53,2015,"""film"""
"""NOVA""",2016-09-25 09:55:00,"""Praštěná holka""","""02.03.2025 v 13:15 (Paramount …","""Další díl/opakování:02.03.2025…","""film/prastena-holka-cluelless/…",9,38,2016,"""film"""
"""Nova Cinema""",2016-09-26 05:55:00,"""Praštěná holka""","""18.01.2025 v 09:25 (NOVA)20.01…","""Další díl/opakování:18.01.2025…","""film/prastena-holka-cluelless/…",9,39,2016,"""film"""
"""NOVA""",2023-05-20 13:15:00,"""Praštěná holka""","""02.03.2025 v 13:15 (Paramount …","""Další díl/opakování:02.03.2025…","""film/prastena-holka-cluelless/…",5,20,2023,"""film"""
"""Nova Cinema""",2023-05-22 10:40:00,"""Praštěná holka""","""18.01.2025 v 09:25 (NOVA)20.01…","""Další díl/opakování:18.01.2025…","""film/prastena-holka-cluelless/…",5,21,2023,"""film"""


In [132]:
from datetime import datetime

In [194]:
roky = range(2007,2025)
prazdniny = []
for r in roky:
    prazdniny.append({'zacatek':datetime(year=r,month=7,day=1), 'konec':datetime(year=r,month=8,day=31)})

prazdniny = pl.DataFrame(prazdniny).with_columns(pl.col('zacatek').dt.week()).with_columns(pl.col('konec').dt.week())
prazdniny

df_prazdniny = []
for x in range(1,54):
    df_prazdniny.append({'tyden':x, 'prazdniny':len(prazdniny.filter((pl.col('zacatek') <= x) & (pl.col('konec') >= x)))})

df_prazdniny = pl.DataFrame(df_prazdniny).with_columns(pl.col('prazdniny') / len(roky)).filter(pl.col("prazdniny") != 0).with_columns(
    pl.col("tyden").min().alias("zacatek")
).with_columns(
    pl.col("tyden").max().alias("konec")
)
df_prazdniny

tyden,prazdniny,zacatek,konec
i64,f64,i64,i64
26,0.555556,26,36
27,1.0,26,36
28,1.0,26,36
29,1.0,26,36
30,1.0,26,36
31,1.0,26,36
32,1.0,26,36
33,1.0,26,36
34,1.0,26,36


In [134]:
prazdniny.filter((pl.col('zacatek') <= 26) & (pl.col('konec') >= 26))

zacatek,konec
i8,i8
26,35
26,35
26,35
26,35
26,35
26,35
26,35
26,35
26,35


In [135]:
df_prazdniny

tyden,prazdniny,zacatek,konec
i64,f64,i64,i64
26,0.555556,26,36
27,1.0,26,36
28,1.0,26,36
29,1.0,26,36
30,1.0,26,36
31,1.0,26,36
32,1.0,26,36
33,1.0,26,36
34,1.0,26,36


In [136]:
terminy = alt.Chart(
    df_prazdniny
).mark_rect(color="pink"
).encode(
    alt.X('zacatek:Q'),
    alt.X2('konec:Q'),
    alt.Opacity('prazdniny:Q')
)

terminy

alt.Chart(...)

In [137]:
df.filter(pl.col("nazev").str.contains("Lepší už"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""NOVA""",2015-07-05 13:00:00,"""Lepší už to nebude""","""31.08.2024 v 13:25 (NOVA)01.09…","""Další díl/opakování:31.08.2024…","""film/lepsi-uz-to-nebude-as-goo…",7,27,2015,"""film"""
"""Nova Cinema""",2015-07-06 10:50:00,"""Lepší už to nebude""","""31.08.2024 v 13:25 (NOVA)01.09…","""Další díl/opakování:31.08.2024…","""film/lepsi-uz-to-nebude-as-goo…",7,28,2015,"""film"""
"""NOVA""",2016-06-04 14:30:00,"""Lepší už to nebude""","""31.08.2024 v 13:25 (NOVA)01.09…","""Další díl/opakování:31.08.2024…","""film/lepsi-uz-to-nebude-as-goo…",6,22,2016,"""film"""
"""Nova Cinema""",2016-06-05 09:50:00,"""Lepší už to nebude""","""31.08.2024 v 13:25 (NOVA)01.09…","""Další díl/opakování:31.08.2024…","""film/lepsi-uz-to-nebude-as-goo…",6,22,2016,"""film"""
"""NOVA""",2017-07-22 14:45:00,"""Lepší už to nebude""","""31.08.2024 v 13:25 (NOVA)01.09…","""Další díl/opakování:31.08.2024…","""film/lepsi-uz-to-nebude-as-goo…",7,29,2017,"""film"""
"""Nova Cinema""",2017-07-23 12:30:00,"""Lepší už to nebude""","""31.08.2024 v 13:25 (NOVA)01.09…","""Další díl/opakování:31.08.2024…","""film/lepsi-uz-to-nebude-as-goo…",7,29,2017,"""film"""
"""Nova Cinema""",2017-12-26 20:00:00,"""Lepší už to nebude""","""31.08.2024 v 13:25 (NOVA)01.09…","""Další díl/opakování:31.08.2024…","""film/lepsi-uz-to-nebude-as-goo…",12,52,2017,"""film"""
"""Nova Cinema""",2017-12-27 14:20:00,"""Lepší už to nebude""","""31.08.2024 v 13:25 (NOVA)01.09…","""Další díl/opakování:31.08.2024…","""film/lepsi-uz-to-nebude-as-goo…",12,52,2017,"""film"""
"""Cinemax""",2018-01-01 20:00:00,"""Lepší už to nebude""","""31.08.2024 v 13:25 (NOVA)01.09…","""Další díl/opakování:31.08.2024…","""film/lepsi-uz-to-nebude-as-goo…",1,1,2018,"""film"""


In [204]:
obsah = alt.Chart(
    graf
).mark_circle(
).encode(
    alt.X('tyden:Q'),
    alt.Y('nazev:N', sort=vyber_strojovy),
    alt.Size('len:Q'),
    alt.Color('kategorie:N')
)

alt.layer(terminy, obsah)

alt.LayerChart(...)

## (dis)kontinuita

In [140]:
df.filter(pl.col('nazev').str.contains('Krotitel')).sample(10)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""Cinemax2""",2018-02-13 11:45:00,"""Krotitelé duchů""","""26.01.2025 v 07:45 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ii-ghostb…",2,7,2018,"""film"""
"""NOVA""",2016-07-07 01:05:00,"""Krotitelé duchů 2""","""09.04.2025 v 12:05 (HBO2)""","""Další díl/opakování:09.04.2025…","""film/krotitele-duchu-ii-ghostb…",7,27,2016,"""film"""
"""Cinemax""",2017-09-22 13:05:00,"""Krotitelé duchů""","""26.01.2025 v 06:00 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ghostbust…",9,38,2017,"""film"""
"""Cinemax""",2018-01-14 13:10:00,"""Krotitelé duchů""","""26.01.2025 v 07:45 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ii-ghostb…",1,2,2018,"""film"""
"""Cinemax""",2018-05-05 12:45:00,"""Krotitelé duchů""","""26.01.2025 v 06:00 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ghostbust…",5,18,2018,"""film"""
"""Cinemax""",2017-08-19 06:00:00,"""Krotitelé duchů""","""26.01.2025 v 06:00 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ghostbust…",8,33,2017,"""film"""
"""Cinemax""",2017-09-22 14:50:00,"""Krotitelé duchů""","""26.01.2025 v 07:45 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ii-ghostb…",9,38,2017,"""film"""
"""Cinemax""",2017-09-22 04:10:00,"""Krotitelé duchů""","""26.01.2025 v 07:45 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ii-ghostb…",9,38,2017,"""film"""
"""Cinemax""",2018-11-23 07:45:00,"""Krotitelé duchů""","""26.01.2025 v 07:45 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ii-ghostb…",11,47,2018,"""film"""


In [141]:
df.select(pl.col('nazev')).sample(20)

nazev
str
"""Na krok od nebe"""
"""Žiješ jenom 2x"""
"""Dnes neordinuji"""
"""Mistři zastavárny XII"""
"""Samorost"""
"""Kung-fu mela"""
"""Námořní vyšetřovací služba L. …"
"""Návrat komisaře Rexe"""
"""McLeodovy dcery"""


In [142]:
def calculate_ratio(current_row: pl.Series, previous_row: pl.Series) -> float:
    current_items = set(current_row)
    previous_items = set(previous_row)

    if not previous_items:
        return 0.0

    common_items = current_items.intersection(previous_items)
    ratio = len(common_items) / len(previous_items)
    return ratio


In [143]:
vyvoj_ct1 = df.filter(pl.col('kanal') == 'ČT1').group_by(['rok','tyden']).agg(pl.col("nazev")).sort(by=['rok','tyden'])

In [144]:
ratios = []
for i in range(1, len(vyvoj_ct1)):
    current_row = vyvoj_ct1["nazev"][i]
    previous_row = vyvoj_ct1["nazev"][i - 1]
    ratio = calculate_ratio(current_row, previous_row)
    ratios.append(ratio)

In [145]:
ratios.insert(0, float('nan'))

In [146]:
vyvoj_ct1 = vyvoj_ct1.with_columns(pl.Series("ratio", ratios))

In [147]:
vyvoj_ct1.filter(pl.col('rok').is_between(2008,2024)).group_by('tyden').agg(pl.col('ratio').median())

tyden,ratio
i8,f64
20,0.636809
52,0.336848
14,0.659098
12,0.695493
50,0.608269
41,0.670068
22,0.680707
51,0.462018
32,0.660256


In [148]:
alt.Chart(
    vyvoj_ct1.filter(pl.col('rok').is_between(2008,2024)).group_by('tyden').agg(pl.col('ratio').median()), width=800
).mark_bar(
).encode(
    alt.X('tyden:Q'),
    alt.Y('ratio:Q')
)

alt.Chart(...)

In [149]:
def calculate_ratio(current_row: pl.Series, previous_row: pl.Series) -> float:
    current_items = set(current_row)
    previous_items = set(previous_row)

    if not previous_items:
        return 0.0

    common_items = current_items.intersection(previous_items)
    ratio = len(common_items) / len(previous_items)
    return ratio
